# **Pruning**
We work on CIFAR-10 dataset for weight pruning – much faster for retraining than ImageNet – trainable from scratch within reasonable time 

* **The CIFAR-10 dataset**

    Consists of 60000 32x32 color images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

* **Aim: prune parameters while keeping testing accuracy for 10 classes**

  Step 1: Get trained model – Train from scratch 

    * Tensorflow CIFAR-10 example
    * PyTorch (reference) – 
       * CIFAR-10: https://github.com/kuangliu/pytorch-cifar 
       * PASCAL VOC: https://github.com/xuzheyuan624/yolov3-pytorch

  Step 2: Implement iterative pruning 
    * Apply magnitude-based pruning as basic approach – You are welcome to go further
  
  Step 3: Analyze pruning result 
    *  tradeoff between parameter amount and accuracy

#Setup Enviornment

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
import numpy as np
import os
import keras
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Lambda 
from keras.layers.normalization import BatchNormalization
from keras import layers
from keras import models
from keras import activations
from keras import backend as K
from keras import losses
import cv2
from google.colab.patches import cv2_imshow
import scipy as sp
from scipy import signal
from numpy import linalg as LA
import matplotlib.pyplot as plt

import math
import timeit
import shutil

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from torchvision.datasets import ImageNet
from torch.utils.data import DataLoader


import torch
from torchvision import transforms
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
from keras.datasets import cifar10
from keras.utils import to_categorical

##load cifar10

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):

  # Load the raw CIFAR-10 data
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()

  # Normalize the data: subtract the mean image
  train_mean_image = np.mean(x_train, axis=0)
  test_mean_image = np.mean(x_test, axis=0)

  X_train = x_train - train_mean_image
  X_test = x_test - test_mean_image


  # # Subsample the data
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask].reshape(-1)

  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask].reshape(-1)

  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask].reshape(-1)

  return X_train, y_train, X_val, y_val, X_test, y_test

## contruct model

In [ ]:
def fire_variables(i, input_size, out_chan1, out_chan2, out_chan3):
  #conv2d, k=1
  f1 = tf.get_variable("Wconv2d_1_"+ str(i), shape=[1, 1, input_size, out_chan1])
  f2 = tf.get_variable("bconv2d_1_"+ str(i), shape=[out_chan1])
  #Relu
  #conv2d, k=1
  f3 = tf.get_variable("Wconv2d_2_"+ str(i), shape=[1, 1, out_chan1, out_chan2])
  f4 = tf.get_variable("bconv2d_2_"+ str(i), shape=[out_chan2])
  #Relu
  #conv2d, k=3
  f5 = tf.get_variable("Wconv2d_3_"+ str(i), shape=[3, 3, out_chan1, out_chan3])
  f6 = tf.get_variable("bconv2d_3_"+ str(i), shape=[out_chan3])
  #Relu
  return f1, f2, f3, f4, f5, f6

def fire_forward(x, weight_F):

    x = tf.nn.conv2d(x, weight_F[0], strides = 1, padding = 'SAME') + weight_F[1]
    x = tf.nn.relu(x)
    #1x1 filter conv2d
    a1 = tf.nn.conv2d(x, weight_F[2], strides = 1, padding = 'VALID') + weight_F[3]
    h1 = tf.nn.relu(a1)
    #3x3 filter conv2d
    a2 = tf.nn.conv2d(x, weight_F[4], strides = 1, padding = 'SAME') + weight_F[5]
    h2 = tf.nn.relu(a2)

    return tf.concat([h1,h2],3)





def model(X,y):

    # define our weights (e.g. init_two_layer_convnet)    
    # setup variables --------------------------------------------------------
    with tf.variable_scope("", reuse=tf.AUTO_REUSE):
      Wconv2d1 = tf.get_variable("Wconv1", shape=[3, 3, 3, 64])
      bconv2d1 = tf.get_variable("bconv1", shape=[64])

      F1 = fire_variables(1, 64, 16, 64, 64)
      F2 = fire_variables(2, 128, 16, 64, 64)

      F3 = fire_variables(3, 128, 32, 128, 128)
      F4 = fire_variables(4, 256, 32, 128, 128)

      F5 = fire_variables(5, 256, 48, 192, 192)
      F6 = fire_variables(6, 384, 48, 192, 192)
      F7 = fire_variables(7, 384, 64, 256, 256)
      F8 = fire_variables(8, 512, 64, 256, 256)

      Wconv2d2 = tf.get_variable("Wconv2", shape=[1, 1, 512, 10])
      bconv2d2 = tf.get_variable("bconv2", shape=[10])

    # define our graph (e.g. two_layer_convnet)-------------------------------
    a1 = tf.nn.conv2d(X, Wconv2d1, strides = 2, padding = 'VALID') + bconv2d1
    h1 = tf.nn.relu(a1)
    p1 = tf.nn.max_pool2d(h1, ksize = 3, strides = 2, padding = 'VALID')
    
    f1 = fire_forward(p1, F1)
    f2 = fire_forward(f1, F2)
    p2 = tf.nn.max_pool2d(f2, ksize = 3, strides = 1, padding = 'VALID')

    f3 = fire_forward(p2, F3)
    f4 = fire_forward(f3, F4)
    p3 = tf.nn.max_pool2d(f4, ksize = 3, strides = 1, padding = 'VALID')

    f5 = fire_forward(p3, F5)
    f6 = fire_forward(f5, F6)
    f7 = fire_forward(f6, F7)
    f8 = fire_forward(f7, F8)

    a2 = tf.nn.conv2d(f8, Wconv2d2, strides = 1, padding='SAME') + bconv2d2
    y_out = tf.nn.avg_pool(a2, ksize = 3, strides = 1, padding = 'VALID')
    y_out = tf.reshape(y_out,[-1,10])
    
    return y_out

##prune

In [ ]:
def prune_param(sess, percentage = 0.6, printing=False):
    all_variables = sess.graph.get_collection('variables')
    masks = []
    pers = []

    for layers in all_variables[1:]:
      # print(layers.name)    
      if ('Adam' not in layers.name) and ('beta' not in layers.name):
        # ================================================================================================================================    
          var_np = layers.eval(session=sess)
          # threshold = np.max(np.abs(var_np))
          per = 0.0 
          threshold = np.sort(np.unique(np.abs(var_np)),axis=None)[::-1][:-1]
          i = int(np.floor(len(threshold)*percentage)) - 1

          # construct pruning mask      
          while (i < len(threshold)) and (per < percentage-0.000000001):          #loop to calculate initial value of threshold reqd. to prune 'percentage' % of the network
              mask = np.abs(var_np) >= threshold[i]                               #bigger = 1, smaller = 0
              mask = mask.astype(np.float32)
              zeros = np.abs(var_np) == 0.0
              per = np.sum(mask)/(np.prod(mask.shape) - np.sum(zeros))         #calc. percentage of parameters bigger than 1
              i += 1
          
          # test if the mask is valid
          if i == len(threshold):
            mask = np.abs(var_np) != 0.0 
            mask = mask.astype(np.float32)
            per = 1.0
            if printing:
              print(layers.name, '    %.2f preserved(%.d)'%(per*100,percentage*100), 'NO PRUNING')

          else:
            if printing:
              print(layers.name, '    %.2f preserved(%.d)'%(per*100,percentage*100), 'Vth:%.4f'%threshold[i-1])


          new_var_np = mask * var_np

          with tf.variable_scope("", reuse=True):
            x = tf.get_variable(layers.name[:-2])  
          prune_op = x.assign(new_var_np)
          sess.run(prune_op)

          masks.append(mask)
          pers.append(per)
          # print(per)

    pers = np.array(pers)
    print('    average %.2f'%(np.sum(pers)/len(pers)*100), '% is preserved,', len(pers), 'masks')
    return masks

In [ ]:
def get_prune_op(sess, train_gradient, masks):
    all_variables = sess.graph.get_collection('variables')
    prune_gradient = []

    for i in range(len(masks)): 
      mask = masks[i]
      mask_tensor = tf.convert_to_tensor(mask)
      v = train_gradient[i]
      new_gradient = (tf.multiply(v[0], mask_tensor), v[1])
      prune_gradient.append(new_gradient)

    return prune_gradient

In [ ]:
def get_prune_op1(sess, train_gradient, percentage=0.1, printing=False, pruning=False):
    all_variables = sess.graph.get_collection('variables')
    
    print('>>> pruning g...')
    masks = []
    prune_gradient = []
    pers = []
 
    for layers in all_variables[1:]:
      # print(layers.name)    
      if ('Adam' not in layers.name) and ('beta' not in layers.name):

        # ================================================================================================================================    
        var_np = layers.eval(session=sess)
        threshold = np.max(np.abs(var_np))
        per = 0.0 

        # construct pruning mask      
        while (threshold > 0) and (per < percentage):                        #loop to calculate initial value of threshold reqd. to prune 'percentage' % of the network
            mask = np.abs(var_np) >= threshold                               #bigger = 1, smaller = 0
            mask = mask.astype(np.float32)
            zeros = np.abs(var_np) == 0.0
            per = np.sum(mask)/(np.prod(mask.shape) - np.sum(zeros))         #calc. percentage of parameters bigger than 1
            threshold -= 0.0001
        
        # test if the mask is valid
        if threshold == 0.0:
          mask = np.abs(var_np) == 0.0 
          mask = mask.astype(np.float32)
          if printing:
            print(layers.name, '    %.2f preserved(%.d)'%(per*100,percentage*100), 'NO PRUNING')

        else:
          if printing:
            print(layers.name, '    %.2f preserved(%.d)'%(per*100,percentage*100), 'Vth:%.4f'%threshold)

        masks.append(mask)
        pers.append(per)

        # ================================================================================================================================
        # apply mask to gradients and parameter pruning by sess.run(prune_op)
        mask_tensor = tf.convert_to_tensor(mask)
        for v in train_gradient:
          if layers.name in v[1].name:
            new_gradient = (tf.multiply(v[0], mask_tensor), v[1])
            prune_gradient.append(new_gradient)
    
    pers = np.array(pers)
    print('    average %.2f'%(np.sum(pers)/len(pers)*100), '% is preserved')
    return prune_gradient, masks

In [ ]:
def get_mask(sess, percentage=0.1, printing=False, pruning=False ):
  all_variables = sess.graph.get_collection('variables')

  masks = []
  pers = []

  #pruining all trainable layers
  for layers in all_variables[1:]:
    # print(layers.name)    
    if ('Adam' not in layers.name) and ('beta' not in layers.name):    
      var_np = layers.eval(session=sess)
      threshold = np.max(np.abs(var_np))
      per = 0.0 

      # construct pruning mask      
      while (threshold > 0) and (per < percentage):                        #loop to calculate initial value of threshold reqd. to prune 'percentage' % of the network
          mask = np.abs(var_np) >= threshold                               #bigger = 1, smaller = 0
          mask = mask.astype(np.float32)
          zeros = np.abs(var_np) == 0.0
          per = np.sum(mask)/(np.prod(mask.shape) - np.sum(zeros))         #calc. percentage of parameters bigger than 1
          threshold -= 0.0001
      
      # test if the mask is valid
      if threshold == 0.0:
        mask = np.abs(var_np) == 0.0 
        mask = mask.astype(np.float32)
        if printing:
          print(layers.name, '    %.2f preserved(%.d)'%(per*100,percentage*100), 'NO PRUNING')

      else:
        if printing:
          print(layers.name, '    %.2f preserved(%.d)'%(per*100,percentage*100), 'Vth:%.4f'%threshold)

      masks.append(mask)
      pers.append(per)

  return masks

# MODEL

In [ ]:
class Model:
    def __init__(self):
        # gpu option
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True 
        self.sess = tf.Session(config=config)
        self.masks = None
        self.the_percentage = 0.7
    
    def load_cifar10(self):        
        # Invoke the above function to get our data.
        self.X_train, self.y_train, self.X_val, self.y_val, self.X_test, self.y_test = get_CIFAR10_data()
        print('Train data shape: ', self.X_train.shape)
        print('Train labels shape: ', self.y_train.shape)
        print('Validation data shape: ', self.X_val.shape)
        print('Validation labels shape: ', self.y_val.shape)
        print('Test data shape: ', self.X_test.shape)
        print('Test labels shape: ', self.y_test.shape)
        
    def construct_model(self, lr=5e-4, ckpt_dir=None, iteration=None, reset=False):
        # Define inputs
        print('Constructing Model...')
        self.X = tf.placeholder(tf.float32, [None, 32, 32, 3])
        self.y = tf.placeholder(tf.int64, [None])
        self.is_training = tf.placeholder(tf.bool)
        self.global_step = tf.train.get_or_create_global_step()
        
        if ckpt_dir == None:
          self.iteration = 0
          self.last_ckpt_dir = './ckpt'
          self.current_ckpt_dir = './ckpt'
          print('    >>> Current checkpoint:',self.current_ckpt_dir)
        else:
          self.current_ckpt_dir = './ckpt' + str(iteration)
          self.iteration = iteration
          print('    >>> Model: last checkpoint:',self.last_ckpt_dir,', current checkpoint:',self.current_ckpt_dir)

        self.y_out = model(self.X,self.y)
        
        # Define our loss
        total_loss = tf.losses.softmax_cross_entropy(tf.one_hot(self.y,10),logits=self.y_out)
        self.mean_loss = tf.reduce_mean(total_loss)

        # Define our optimizer
        self.optimizer = tf.train.AdamOptimizer(lr) # select optimizer and set learning rate
            
        # split train_step = optimizer.minimize(self.mean_loss)
        train_gradient = self.optimizer.compute_gradients(self.mean_loss)
        
        # initialize or load model parameters
        self.saver = tf.train.Saver(max_to_keep=self.iteration+10)

        # restore the model from last time
        if ckpt_dir is not None:
            self.saver.restore(self.sess, tf.train.latest_checkpoint(self.last_ckpt_dir))
            # print('Pre-trained model restored from %s' % (self.last_ckpt_dir))
        else:
            self.sess.run(tf.global_variables_initializer())
            print('Initialize variables')

        if self.masks is not None:
          print('Start Pruning...')
          for c in range(20):
            self.masks = prune_param(self.sess, percentage= self.the_percentage)
            _, total_correct = self.run_model(self.sess,self.y_out,self.mean_loss,self.X_test,self.y_test, epochs=1,batch_size=64, saving=False, printing=False)

            if (self.total_correct - total_correct) > 0.005 and (self.the_percentage <= 1.0):
              self.saver.restore(self.sess, tf.train.latest_checkpoint(self.last_ckpt_dir))
              self.the_percentage += 0.05
              print('    (acc: %.3f->%.3f) Increasimg preservation to %.2f of parameters'%( self.total_correct*100, total_correct*100,self.the_percentage*100))
            else:
              print('    acc: %.3f->%.3f, PRUNING ACCEPTED'%(self.total_correct*100, total_correct*100))
              break

          prune_gradient = get_prune_op(self.sess, train_gradient, self.masks)

        else:
          prune_gradient = train_gradient
          self.masks = False

        #initialize the default
        self.the_percentage = 0.7
        print('Storing pruned graph, initializing, and restoring...')

        # save pruned parameters
        tmp_dir = '__tmp__'

        if not os.path.exists(tmp_dir):
            os.makedirs(tmp_dir)

        self.saver.save(self.sess, os.path.join(tmp_dir, 'pruned_model.ckpt'))

        # define gradients and initialize optimizer parameters
        self.train_op = self.optimizer.apply_gradients(prune_gradient, global_step=self.global_step)
        self.sess.run(tf.global_variables_initializer())
        
        if reset:
          all_variables = self.sess.graph.get_collection('variables')

          for i in range(len(all_variables[1:])):
            layers = all_variables[i+1]
            if ('Adam' not in layers.name) and ('beta' not in layers.name):
                var_np = layers.eval(session=self.sess)
                new_var_np = self.masks[i] * var_np
                with tf.variable_scope("", reuse=True):
                  x = tf.get_variable(layers.name[:-2])  
                prune_op = x.assign(new_var_np)
                self.sess.run(prune_op)
        else:
          self.saver.restore(self.sess, tf.train.latest_checkpoint(tmp_dir))

        # reload pruned parameters
        if os.path.exists(tmp_dir):
            shutil.rmtree(tmp_dir)        
        
        
    def run_model(self, sess, predict, loss_val, Xd, yd, epochs=1, batch_size=64, print_every=100, save_every=1, train_op=None, plot_losses=False, saving=True, printing=True):
        # have tensorflow compute accuracy
        correct_prediction = tf.equal(tf.argmax(predict,1), self.y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        # shuffle indicies
        train_indicies = np.arange(Xd.shape[0])
        np.random.shuffle(train_indicies)

        training_now = train_op is not None
        
        # setting up variables we want to compute (and optimizing)
        # if we have a training function, add that to things we compute
        variables = [loss_val,correct_prediction,accuracy]
        if training_now:
            variables[-1] = train_op

        # counter 
        iter_cnt = 0
        for e in range(epochs):
            # keep track of losses and accuracy
            correct = 0
            losses = []

            # make sure we iterate over the dataset once
            for i in range(int(math.ceil(Xd.shape[0]/batch_size))): # TODO: ceil?? Is this right?!
                # generate indicies for the batch
                start_idx = (i*batch_size)%Xd.shape[0]
                idx = train_indicies[start_idx:start_idx+batch_size]
                
                # create a feed dictionary for this batch
                feed_dict = {self.X: Xd[idx,:],
                             self.y: yd[idx],
                             self.is_training: training_now }
                # get batch size
                actual_batch_size = yd[idx].shape[0]
                
                # have tensorflow compute loss and correct predictions
                # and (if given) perform a training step
                loss, corr, _ = sess.run(variables,feed_dict=feed_dict)
                
                # aggregate performance stats
                losses.append(loss*actual_batch_size)
                correct += np.sum(corr)
                
                # print every now and then
                if training_now and (iter_cnt % print_every) == 0:
                    print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                          .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
                iter_cnt += 1

            total_correct = correct/Xd.shape[0]
            total_loss = np.sum(losses)/Xd.shape[0]

            if printing==True:
                print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
                      .format(total_loss,total_correct,e+1))
            
            if plot_losses:
                plt.plot(losses)
                plt.grid(True)
                plt.title('Epoch {} Loss'.format(e+1))
                plt.xlabel('minibatch number')
                plt.ylabel('minibatch loss')
                plt.show()

            # Save the model checkpoint periodically.
            if e % save_every == 0 and saving == True:
                checkpoint_path = os.path.join(self.current_ckpt_dir, 'model.ckpt')
                self.saver.save(sess, checkpoint_path, global_step=int(e+1))

        return total_loss,total_correct 
        
    def train(self, E=10):
        print('Training')
        self.run_model(self.sess,self.y_out,self.mean_loss,self.X_train,self.y_train,epochs=E,batch_size=64,print_every=50,save_every=1,train_op=self.train_op,plot_losses=False)
        print('Validation')
        self.run_model(self.sess,self.y_out,self.mean_loss,self.X_val,self.y_val, epochs=1,batch_size=64, saving=False)
        print('Testing')
        total_loss, total_correct = self.run_model(self.sess,self.y_out,self.mean_loss,self.X_test,self.y_test, epochs=1,batch_size=64, saving=False)
        self.total_correct = total_correct
        return total_loss, total_correct
    
    
    def check(self, printing=False, accept=True):
      zeros = []
      all_variables = self.sess.graph.get_collection('variables')
      print('>>> Checking the layers...',len(all_variables))
      #analyze each layer
      for layers in all_variables[1:]:
        if ('Adam' not in layers.name) and ('beta' not in layers.name):    
          var_np = layers.eval(session=self.sess)
          temp = var_np == 0.0
          p = np.sum(temp)/np.prod(var_np.shape)
          zeros.append(100-p*100)
          
          if printing:
            print(layers.name, '\tmin/max = %.6f/%.6f'%(np.min(var_np),np.max(var_np)), '{0:.2f}'.format(p*100),'% is 0')

      
      # self.last_ckpt_dir= self.current_ckpt_dir
      # self.total_correct = 0.15
      
      if accept:
        self.last_ckpt_dir = self.current_ckpt_dir
      
      zeros = np.array(zeros)
      print('    Downsized to %.2f'%(np.sum(zeros)/len(zeros)),'% of the original') 


    def clear_graph(self):
        print('Clearing graph...')
        self.sess.close()
        tf.keras.backend.clear_session()
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True 
        self.sess = tf.Session(config=config)

    def print_layer(self):
        print('    Showing a layer in the model...\n', all_variables[2].eval(session=self.sess))

    def definition(self, ckptdir, total_correct, masks=None, per= 0.7):
        self.last_ckpt_dir = ckptdir
        self.total_correct = total_correct
        self.masks = masks
        self.the_percentage = per

#Evaluation

In [ ]:
%cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307

m = Model()
m.load_cifar10()
for i in range(1):
  print('\n')
  print('<'*20, '\t    Initial Round', '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4)
  _, accuracy = m.train(10)
  m.check()

for i in range(1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i)
  _, accuracy1 = m.train(10+i)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1
  

##Parts 1

In [ ]:
for i in range(1,2,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(10)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1
  



<<<<<<<<<<<<<<<<<<<< 	    ROUND 1  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt0 , current checkpoint: ./ckpt1
INFO:tensorflow:Restoring parameters from ./ckpt0/model.ckpt-10
    average 70.47 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt0/model.ckpt-10
    (acc: 67.740->61.350) Increasimg preservation to 75.00 of parameters
    average 75.37 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt0/model.ckpt-10
    (acc: 67.740->64.380) Increasimg preservation to 80.00 of parameters
    average 80.26 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt0/model.ckpt-10
    (acc: 67.740->65.740) Increasimg preservation to 85.00 of parameters
    average 85.47 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt0/model.ckpt-10
    (acc: 67.740->66.370) Increasimg preservation to 90.00 of parameters
    average 90.48 % is preserved, 52 masks
   

In [ ]:
for i in range(2,3,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(10)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1
  



<<<<<<<<<<<<<<<<<<<< 	    ROUND 2  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt1 , current checkpoint: ./ckpt2
INFO:tensorflow:Restoring parameters from ./ckpt1/model.ckpt-10
    average 70.41 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt1/model.ckpt-10
    (acc: 64.460->56.060) Increasimg preservation to 75.00 of parameters
    average 75.62 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt1/model.ckpt-10
    (acc: 64.460->58.720) Increasimg preservation to 80.00 of parameters
    average 80.16 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt1/model.ckpt-10
    (acc: 64.460->62.080) Increasimg preservation to 85.00 of parameters
    average 85.42 % is preserved, 52 masks
    acc: 64.460->64.060, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
INFO:tensorflow:Restoring parameters from __tmp__/pruned_model.ckpt
Training
Epoch 1, Ov

In [ ]:
for i in range(3,4,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(10)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1
  



<<<<<<<<<<<<<<<<<<<< 	    ROUND 3  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt2 , current checkpoint: ./ckpt3
INFO:tensorflow:Restoring parameters from ./ckpt2/model.ckpt-10
    average 70.66 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt2/model.ckpt-10
    (acc: 65.350->42.430) Increasimg preservation to 75.00 of parameters
    average 75.43 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt2/model.ckpt-10
    (acc: 65.350->51.540) Increasimg preservation to 80.00 of parameters
    average 80.75 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt2/model.ckpt-10
    (acc: 65.350->56.790) Increasimg preservation to 85.00 of parameters
    average 85.58 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt2/model.ckpt-10
    (acc: 65.350->60.420) Increasimg preservation to 90.00 of parameters
    average 90.71 % is preserved, 52 masks
INF

In [ ]:
for i in range(4,5,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(13)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 4  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt3 , current checkpoint: ./ckpt4
INFO:tensorflow:Restoring parameters from ./ckpt3/model.ckpt-10
    average 70.83 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt3/model.ckpt-10
    (acc: 65.820->31.850) Increasimg preservation to 75.00 of parameters
    average 75.55 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt3/model.ckpt-10
    (acc: 65.820->46.020) Increasimg preservation to 80.00 of parameters
    average 80.56 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt3/model.ckpt-10
    (acc: 65.820->54.360) Increasimg preservation to 85.00 of parameters
    average 85.80 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt3/model.ckpt-10
    (acc: 65.820->60.460) Increasimg preservation to 90.00 of parameters
    average 90.72 % is preserved, 52 masks
INF

In [ ]:
for i in range(5,6,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(13)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 5  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt4 , current checkpoint: ./ckpt5
INFO:tensorflow:Restoring parameters from ./ckpt4/model.ckpt-13
    average 70.84 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt4/model.ckpt-13
    (acc: 62.560->58.700) Increasimg preservation to 75.00 of parameters
    average 75.55 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt4/model.ckpt-13
    (acc: 62.560->60.670) Increasimg preservation to 80.00 of parameters
    average 80.56 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt4/model.ckpt-13
    (acc: 62.560->60.900) Increasimg preservation to 85.00 of parameters
    average 85.79 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt4/model.ckpt-13
    (acc: 62.560->61.110) Increasimg preservation to 90.00 of parameters
    average 90.72 % is preserved, 52 masks
   

In [ ]:
for i in range(6,7,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(13)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 6  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt5 , current checkpoint: ./ckpt6
INFO:tensorflow:Restoring parameters from ./ckpt5/model.ckpt-13
    average 70.71 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt5/model.ckpt-13
    (acc: 64.990->42.310) Increasimg preservation to 75.00 of parameters
    average 75.30 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt5/model.ckpt-13
    (acc: 64.990->56.710) Increasimg preservation to 80.00 of parameters
    average 80.56 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt5/model.ckpt-13
    (acc: 64.990->61.710) Increasimg preservation to 85.00 of parameters
    average 85.60 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt5/model.ckpt-13
    (acc: 64.990->63.730) Increasimg preservation to 90.00 of parameters
    average 90.40 % is preserved, 52 masks
INF

In [ ]:
for i in range(7,8,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(13)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 7  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt6 , current checkpoint: ./ckpt7
INFO:tensorflow:Restoring parameters from ./ckpt6/model.ckpt-13
    average 70.75 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt6/model.ckpt-13
    (acc: 64.010->42.020) Increasimg preservation to 75.00 of parameters
    average 75.56 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt6/model.ckpt-13
    (acc: 64.010->50.580) Increasimg preservation to 80.00 of parameters
    average 80.70 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt6/model.ckpt-13
    (acc: 64.010->59.440) Increasimg preservation to 85.00 of parameters
    average 86.05 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt6/model.ckpt-13
    (acc: 64.010->61.720) Increasimg preservation to 90.00 of parameters
    average 90.93 % is preserved, 52 masks
   

In [ ]:
for i in range(8,9,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(13)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 8  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt7 , current checkpoint: ./ckpt8
INFO:tensorflow:Restoring parameters from ./ckpt7/model.ckpt-13
    average 70.86 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt7/model.ckpt-13
    (acc: 63.350->52.440) Increasimg preservation to 75.00 of parameters
    average 75.59 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt7/model.ckpt-13
    (acc: 63.350->57.880) Increasimg preservation to 80.00 of parameters
    average 80.69 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt7/model.ckpt-13
    (acc: 63.350->60.410) Increasimg preservation to 85.00 of parameters
    average 85.85 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt7/model.ckpt-13
    (acc: 63.350->61.950) Increasimg preservation to 90.00 of parameters
    average 90.89 % is preserved, 52 masks
INF

In [ ]:
for i in range(10,11,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=5e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(13)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 10  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt9 , current checkpoint: ./ckpt10
INFO:tensorflow:Restoring parameters from ./ckpt9/model.ckpt-13
    average 70.76 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt9/model.ckpt-13
    (acc: 60.430->51.980) Increasimg preservation to 75.00 of parameters
    average 75.54 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt9/model.ckpt-13
    (acc: 60.430->56.370) Increasimg preservation to 80.00 of parameters
    average 80.57 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt9/model.ckpt-13
    (acc: 60.430->56.350) Increasimg preservation to 85.00 of parameters
    average 86.02 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt9/model.ckpt-13
    (acc: 60.430->59.220) Increasimg preservation to 90.00 of parameters
    average 91.07 % is preserved, 52 masks
I

In [ ]:
for i in range(11,12,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=8e-4, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 11  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt10 , current checkpoint: ./ckpt11
INFO:tensorflow:Restoring parameters from ./ckpt10/model.ckpt-13
    average 70.64 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt10/model.ckpt-13
    (acc: 57.600->51.150) Increasimg preservation to 75.00 of parameters
    average 75.89 % is preserved, 52 masks
    acc: 57.600->58.690, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
Training
Epoch 1, Overall loss = 2 and accuracy of 0.194
Epoch 2, Overall loss = 1.77 and accuracy of 0.301
Epoch 3, Overall loss = 1.56 and accuracy of 0.4
Epoch 4, Overall loss = 1.43 and accuracy of 0.462
Epoch 5, Overall loss = 1.33 and accuracy of 0.507
Epoch 6, Overall loss = 1.24 and accuracy of 0.546
Epoch 7, Overall loss = 1.18 and accuracy of 0.569
Epoch 8, Overall loss = 1.14 and accuracy of 0.59
Epoch 9, Overall loss = 1.1 and 

In [ ]:
for i in range(12,13,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=8e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 12  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt11 , current checkpoint: ./ckpt12
INFO:tensorflow:Restoring parameters from ./ckpt11/model.ckpt-14
    average 70.72 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt11/model.ckpt-14
    (acc: 60.790->53.690) Increasimg preservation to 75.00 of parameters
    average 75.65 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt11/model.ckpt-14
    (acc: 60.790->58.240) Increasimg preservation to 80.00 of parameters
    average 81.26 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt11/model.ckpt-14
    (acc: 60.790->58.740) Increasimg preservation to 85.00 of parameters
    average 85.95 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt11/model.ckpt-14
    (acc: 60.790->60.010) Increasimg preservation to 90.00 of parameters
    average 91.38 % is preserved, 52 m

In [ ]:
%cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307
m = Model()
m.load_cifar10()
accuracy = 0.6276
m.definition('./ckpt12', accuracy)

/content/gdrive/My Drive/UCLA/209AS AI on Chips/Project3/0307
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [ ]:
for i in range(13,14,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=8e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 13  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt12 , current checkpoint: ./ckpt13
INFO:tensorflow:Restoring parameters from ./ckpt12/model.ckpt-14
    average 71.25 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt12/model.ckpt-14
    (acc: 62.760->49.680) Increasimg preservation to 75.00 of parameters
    average 76.24 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt12/model.ckpt-14
    (acc: 62.760->58.030) Increasimg preservation to 80.00 of parameters
    average 81.35 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt12/model.ckpt-14
    (acc: 62.760->60.930) Increasimg preservation to 85.00 of parameters
    average 85.80 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt12/model.ckpt-14
    (acc: 62.760->61.610) Increasimg preservation to 90.00 of parameters
    average 91.18 % is preserved, 52 m

In [ ]:
accuracy = 0.6216
m.definition('./ckpt13', accuracy)
for i in range(14,15,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=8e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 14  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt13 , current checkpoint: ./ckpt14
INFO:tensorflow:Restoring parameters from ./ckpt13/model.ckpt-14
    average 70.92 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt13/model.ckpt-14
    (acc: 62.160->51.910) Increasimg preservation to 75.00 of parameters
    average 75.94 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt13/model.ckpt-14
    (acc: 62.160->57.060) Increasimg preservation to 80.00 of parameters
    average 81.19 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt13/model.ckpt-14
    (acc: 62.160->59.960) Increasimg preservation to 85.00 of parameters
    average 85.73 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt13/model.ckpt-14
    (acc: 62.160->60.170) Increasimg preservation to 90.00 of parameters
    average 91.24 % is preserved, 52 m

In [ ]:
for i in range(15,16,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=8e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 15  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt14 , current checkpoint: ./ckpt15
INFO:tensorflow:Restoring parameters from ./ckpt14/model.ckpt-14
    average 70.92 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt14/model.ckpt-14
    (acc: 62.850->48.040) Increasimg preservation to 75.00 of parameters
    average 75.94 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt14/model.ckpt-14
    (acc: 62.850->54.260) Increasimg preservation to 80.00 of parameters
    average 81.19 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt14/model.ckpt-14
    (acc: 62.850->57.830) Increasimg preservation to 85.00 of parameters
    average 85.74 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt14/model.ckpt-14
    (acc: 62.850->60.680) Increasimg preservation to 90.00 of parameters
    average 91.24 % is preserved, 52 m

In [ ]:
for i in range(16,17,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=8e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 16  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt15 , current checkpoint: ./ckpt16
INFO:tensorflow:Restoring parameters from ./ckpt15/model.ckpt-14
    average 70.83 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt15/model.ckpt-14
    (acc: 63.310->49.210) Increasimg preservation to 75.00 of parameters
    average 75.84 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt15/model.ckpt-14
    (acc: 63.310->56.790) Increasimg preservation to 80.00 of parameters
    average 81.14 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt15/model.ckpt-14
    (acc: 63.310->58.890) Increasimg preservation to 85.00 of parameters
    average 85.70 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt15/model.ckpt-14
    (acc: 63.310->60.700) Increasimg preservation to 90.00 of parameters
    average 91.30 % is preserved, 52 m

## Parts 2

In [ ]:
for i in range(17,18,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=8e-4, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 17  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt16 , current checkpoint: ./ckpt17
INFO:tensorflow:Restoring parameters from ./ckpt16/model.ckpt-14
    average 71.40 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt16/model.ckpt-14
    (acc: 63.050->34.320) Increasimg preservation to 75.00 of parameters
    average 75.95 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt16/model.ckpt-14
    (acc: 63.050->40.530) Increasimg preservation to 80.00 of parameters
    average 81.46 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt16/model.ckpt-14
    (acc: 63.050->50.880) Increasimg preservation to 85.00 of parameters
    average 86.40 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt16/model.ckpt-14
    (acc: 63.050->56.660) Increasimg preservation to 90.00 of parameters
    average 91.75 % is preserved, 52 m

In [ ]:
accuracy = 0.6352
m.definition('./ckpt17', accuracy)
for i in range(18,19,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 18  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt17 , current checkpoint: ./ckpt18
INFO:tensorflow:Restoring parameters from ./ckpt17/model.ckpt-14
    average 71.39 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt17/model.ckpt-14
    (acc: 63.520->38.280) Increasimg preservation to 75.00 of parameters
    average 75.95 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt17/model.ckpt-14
    (acc: 63.520->46.120) Increasimg preservation to 80.00 of parameters
    average 81.46 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt17/model.ckpt-14
    (acc: 63.520->52.650) Increasimg preservation to 85.00 of parameters
    average 86.40 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt17/model.ckpt-14
    (acc: 63.520->58.260) Increasimg preservation to 90.00 of parameters
    average 91.75 % is preserved, 52 m

In [ ]:
for i in range(19,20,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 19  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt18 , current checkpoint: ./ckpt19
INFO:tensorflow:Restoring parameters from ./ckpt18/model.ckpt-14
    average 71.39 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt18/model.ckpt-14
    (acc: 62.960->37.710) Increasimg preservation to 75.00 of parameters
    average 75.95 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt18/model.ckpt-14
    (acc: 62.960->44.730) Increasimg preservation to 80.00 of parameters
    average 81.52 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt18/model.ckpt-14
    (acc: 62.960->53.810) Increasimg preservation to 85.00 of parameters
    average 86.40 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt18/model.ckpt-14
    (acc: 62.960->56.800) Increasimg preservation to 90.00 of parameters
    average 91.76 % is preserved, 52 m

In [ ]:
for i in range(20,21,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 20  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt19 , current checkpoint: ./ckpt20
INFO:tensorflow:Restoring parameters from ./ckpt19/model.ckpt-14
    average 71.39 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt19/model.ckpt-14
    (acc: 63.680->40.850) Increasimg preservation to 75.00 of parameters
    average 75.94 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt19/model.ckpt-14
    (acc: 63.680->46.140) Increasimg preservation to 80.00 of parameters
    average 81.46 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt19/model.ckpt-14
    (acc: 63.680->53.440) Increasimg preservation to 85.00 of parameters
    average 86.40 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt19/model.ckpt-14
    (acc: 63.680->58.360) Increasimg preservation to 90.00 of parameters
    average 91.75 % is preserved, 52 m

In [ ]:
for i in range(21,22,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 21  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt20 , current checkpoint: ./ckpt21
INFO:tensorflow:Restoring parameters from ./ckpt20/model.ckpt-14
    average 71.40 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt20/model.ckpt-14
    (acc: 63.320->37.750) Increasimg preservation to 75.00 of parameters
    average 75.94 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt20/model.ckpt-14
    (acc: 63.320->43.240) Increasimg preservation to 80.00 of parameters
    average 81.47 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt20/model.ckpt-14
    (acc: 63.320->52.280) Increasimg preservation to 85.00 of parameters
    average 86.40 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt20/model.ckpt-14
    (acc: 63.320->58.870) Increasimg preservation to 90.00 of parameters
    average 91.75 % is preserved, 52 m

In [ ]:
for i in range(22,23,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 22  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt21 , current checkpoint: ./ckpt22
INFO:tensorflow:Restoring parameters from ./ckpt21/model.ckpt-14
    average 71.29 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt21/model.ckpt-14
    (acc: 58.380->48.230) Increasimg preservation to 75.00 of parameters
    average 76.22 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt21/model.ckpt-14
    (acc: 58.380->53.250) Increasimg preservation to 80.00 of parameters
    average 81.30 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt21/model.ckpt-14
    (acc: 58.380->56.460) Increasimg preservation to 85.00 of parameters
    average 86.41 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt21/model.ckpt-14
    (acc: 58.380->57.750) Increasimg preservation to 90.00 of parameters
    average 91.77 % is preserved, 52 m

In [ ]:
for i in range(23,24,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 23  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt22 , current checkpoint: ./ckpt23
INFO:tensorflow:Restoring parameters from ./ckpt22/model.ckpt-14
    average 71.22 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt22/model.ckpt-14
    (acc: 61.750->47.170) Increasimg preservation to 75.00 of parameters
    average 75.69 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt22/model.ckpt-14
    (acc: 61.750->53.530) Increasimg preservation to 80.00 of parameters
    average 81.18 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt22/model.ckpt-14
    (acc: 61.750->57.270) Increasimg preservation to 85.00 of parameters
    average 86.51 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt22/model.ckpt-14
    (acc: 61.750->59.760) Increasimg preservation to 90.00 of parameters
    average 91.81 % is preserved, 52 m

## Parts 3

In [ ]:
%cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307

m = Model()
m.load_cifar10()
accuracy = 0.61
m.definition('./ckpt23',accuracy, False)

/content/gdrive/My Drive/UCLA/209AS AI on Chips/Project3/0307
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [ ]:
for i in range(24,25,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 24  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt23 , current checkpoint: ./ckpt24
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from ./ckpt23/model.ckpt-14
    average 71.13 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt23/model.ckpt-14
    (acc: 61.000->40.220) Increasimg preservation to 75.00 of parameters
    average 75.83 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt23/model.ckpt-14
    (acc: 61.000->49.610) Increasimg preservation to 80.00 of parameters
    average 81.34 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt23/model.ckpt-14
    (acc: 61.000->56.250) Increasimg preservation to 85.00 of parameters
    average 86.40 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt23/model.ckpt-14
    (acc: 61.000-

In [ ]:
for i in range(25,26,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 25  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt24 , current checkpoint: ./ckpt25
INFO:tensorflow:Restoring parameters from ./ckpt24/model.ckpt-14
    average 71.18 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt24/model.ckpt-14
    (acc: 62.220->38.980) Increasimg preservation to 75.00 of parameters
    average 75.90 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt24/model.ckpt-14
    (acc: 62.220->50.240) Increasimg preservation to 80.00 of parameters
    average 81.17 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt24/model.ckpt-14
    (acc: 62.220->55.320) Increasimg preservation to 85.00 of parameters
    average 86.27 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt24/model.ckpt-14
    (acc: 62.220->57.850) Increasimg preservation to 90.00 of parameters
    average 91.74 % is preserved, 52 m

In [ ]:
%cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307
m = Model()
m.load_cifar10()

accuracy = 0.6354
m.definition('./ckpt25',accuracy, False, 1.0)

for i in range(26,27,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(18)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 26  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt25 , current checkpoint: ./ckpt26
INFO:tensorflow:Restoring parameters from ./ckpt25/model.ckpt-14
    average 99.99 % is preserved, 52 masks
    acc: 63.540->63.540, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
Training
Epoch 1, Overall loss = 2.02 and accuracy of 0.173
Epoch 2, Overall loss = 1.84 and accuracy of 0.244
Epoch 3, Overall loss = 1.65 and accuracy of 0.355
Epoch 4, Overall loss = 1.53 and accuracy of 0.413
Epoch 5, Overall loss = 1.42 and accuracy of 0.461
Epoch 6, Overall loss = 1.35 and accuracy of 0.494
Epoch 7, Overall loss = 1.29 and accuracy of 0.522
Epoch 8, Overall loss = 1.25 and accuracy of 0.544
Epoch 9, Overall loss = 1.21 and accuracy of 0.562
Epoch 10, Overall loss = 1.17 and accuracy of 0.575
Epoch 11, Overall loss = 1.15 and accuracy of 0.586
Epoch 12, Overall loss = 1.13 and accuracy of 0.596


In [ ]:
# accuracy = 0.6354
# m.definition('./ckpt26',accuracy, False)

for i in range(27,28,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=3e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 27  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt26 , current checkpoint: ./ckpt27
INFO:tensorflow:Restoring parameters from ./ckpt26/model.ckpt-18
    average 71.12 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt26/model.ckpt-18
    (acc: 58.620->49.170) Increasimg preservation to 75.00 of parameters
    average 75.76 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt26/model.ckpt-18
    (acc: 58.620->55.400) Increasimg preservation to 80.00 of parameters
    average 81.31 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt26/model.ckpt-18
    (acc: 58.620->56.500) Increasimg preservation to 85.00 of parameters
    average 86.33 % is preserved, 52 masks
    acc: 58.620->58.760, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
INFO:tensorflow:Restoring parameters from __tmp__/pruned_model.ckpt
Training
Epoc

In [ ]:
%cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307

m = Model()
m.load_cifar10()
accuracy = 0.5988
m.definition('./ckpt27',accuracy, False)

/content/gdrive/My Drive/UCLA/209AS AI on Chips/Project3/0307
170500096/170498071 [==============================] - 6s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [ ]:
for i in range(28,29,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=3e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 28  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt27 , current checkpoint: ./ckpt28
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from ./ckpt27/model.ckpt-14
    average 71.58 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt27/model.ckpt-14
    (acc: 59.880->41.410) Increasimg preservation to 75.00 of parameters
    average 76.44 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt27/model.ckpt-14
    (acc: 59.880->53.370) Increasimg preservation to 80.00 of parameters
    average 80.79 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt27/model.ckpt-14
    (acc: 59.880->56.520) Increasimg preservation to 85.00 of parameters
    average 86.39 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt27/model.ckpt-14
    (acc: 59.880-

##Parts 4


In [ ]:
%cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307
m = Model()
m.load_cifar10()
accuracy = 0.6413
m.definition('./ckpt28',accuracy, False, 0.95)

for i in range(29,30,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=3e-3, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(14)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1

[Errno 2] No such file or directory: '/content/gdrive/My Drive/UCLA/209AS AI on Chips/Project3/0307'
/content


NameError: ignored

In [ ]:
accuracy = 0.639
m.definition('./ckpt29',accuracy, False, 0.9)
for i in range(30,31,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=2.5e-3, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(22)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 30  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt29 , current checkpoint: ./ckpt30
INFO:tensorflow:Restoring parameters from ./ckpt29/model.ckpt-14
    average 91.24 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt29/model.ckpt-14
    (acc: 63.900->63.140) Increasimg preservation to 95.00 of parameters
    average 96.55 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt29/model.ckpt-14
    (acc: 63.900->63.390) Increasimg preservation to 100.00 of parameters
    average 100.00 % is preserved, 52 masks
    acc: 63.900->63.900, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
Training
Epoch 1, Overall loss = 2 and accuracy of 0.194
Epoch 2, Overall loss = 1.82 and accuracy of 0.278
Epoch 3, Overall loss = 1.65 and accuracy of 0.362
Epoch 4, Overall loss = 1.48 and accuracy of 0.446
Epoch 5, Overall loss = 1.38 and accuracy of 0.488

In [ ]:
accuracy = 0.5894
m.definition('./ckpt30',accuracy, False)
for i in range(31,32,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=2.3e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(20)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 31  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt30 , current checkpoint: ./ckpt31
INFO:tensorflow:Restoring parameters from ./ckpt30/model.ckpt-22
Start Pruning...
    average 71.34 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt30/model.ckpt-22
    (acc: 58.940->56.550) Increasimg preservation to 75.00 of parameters
    average 76.91 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt30/model.ckpt-22
    (acc: 58.940->57.380) Increasimg preservation to 80.00 of parameters
    average 80.82 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt30/model.ckpt-22
    (acc: 58.940->57.830) Increasimg preservation to 85.00 of parameters
    average 86.31 % is preserved, 52 masks
    acc: 58.940->58.580, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
INFO:tensorflow:Restoring parameters from __tmp__/pruned_model.c

In [ ]:
%cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307
m = Model()
m.load_cifar10()
accuracy = 0.6051
m.definition('./ckpt31',accuracy, False, 1.0)
for i in range(32,33,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=2e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(18)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1

/content/gdrive/My Drive/UCLA/209AS AI on Chips/Project3/0307
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


<<<<<<<<<<<<<<<<<<<< 	    ROUND 32  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt31 , current checkpoint: ./ckpt32
INFO:tensorflow:Restoring parameters from ./ckpt31/model.ckpt-20
Start Pruning...
    average 100.00 % is preserved, 52 masks
    acc: 60.510->60.510, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
INFO:tensorflow:Restoring parameters from __tmp__/pruned_model.ckpt
Training
Epoch 1, Overall loss = 0.976 and accuracy of 0.651
Epoch 2, Overall loss = 0.955 and accuracy of 0.658
Epoch 3, Overall loss = 0.953 and accuracy of 0.661
Epoch 4, Overall loss = 0.941 and accuracy of 0.662
Epoch 5, Overall loss = 0.942 and accuracy of 0.6

In [ ]:
%cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307
m = Model()
m.load_cifar10()
accuracy = 0.6104
m.definition('./ckpt32',accuracy, False, 0.95)
for i in range(33,34,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1.7e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(18)

  if accuracy - accuracy1 > 0.1:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1

/content/gdrive/My Drive/UCLA/209AS AI on Chips/Project3/0307
170500096/170498071 [==============================] - 4s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


<<<<<<<<<<<<<<<<<<<< 	    ROUND 33  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt32 , current checkpoint: ./ckpt33
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from ./ckpt32/model.ckpt-18
Start Pruning...
    average 96.73 % is preserved, 52 masks
    acc: 61.040->60.650, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
INFO:tensorflow:Restoring parameters from __tmp__/pruned_model.ckpt
Training
Epoch 1, Overall loss = 0.889 and accuracy of 0.682
Epoch 2, Overall loss = 0.876 and accuracy of

In [ ]:
accuracy = 0.57
m.definition('./ckpt33',accuracy, False)
for i in range(34,35,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=2.3e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(19)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 34  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt33 , current checkpoint: ./ckpt34
INFO:tensorflow:Restoring parameters from ./ckpt33/model.ckpt-18
    average 72.69 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt33/model.ckpt-18
    (acc: 57.000->51.800) Increasimg preservation to 75.00 of parameters
    average 76.17 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt33/model.ckpt-18
    (acc: 57.000->52.760) Increasimg preservation to 80.00 of parameters
    average 81.65 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt33/model.ckpt-18
    (acc: 57.000->54.720) Increasimg preservation to 85.00 of parameters
    average 88.04 % is preserved, 52 masks
    acc: 57.000->56.720, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
INFO:tensorflow:Restoring parameters from __tmp__/pruned_model.ckpt
Training
Epoc

##Parts 5

In [ ]:
# #rerun Here
# %cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307
# m = Model()
# m.load_cifar10()

# accuracy = 0.6269
# m.definition('./ckpt34',accuracy, False, 0.8)
for i in range(35,36,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=2e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(15)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 35  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt34 , current checkpoint: ./ckpt35
INFO:tensorflow:Restoring parameters from ./ckpt34/model.ckpt-19
    average 72.36 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt34/model.ckpt-19
    (acc: 59.750->48.600) Increasimg preservation to 75.00 of parameters
    average 76.14 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt34/model.ckpt-19
    (acc: 59.750->50.370) Increasimg preservation to 80.00 of parameters
    average 81.45 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt34/model.ckpt-19
    (acc: 59.750->54.700) Increasimg preservation to 85.00 of parameters
    average 87.47 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt34/model.ckpt-19
    (acc: 59.750->56.390) Increasimg preservation to 90.00 of parameters
    average 91.84 % is preserved, 52 m

In [ ]:
#rerun Here
# accuracy = 0.6245
m.definition('./ckpt35',accuracy, False, 1.0)
for i in range(36,37,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=2e-3, ckpt_dir=True, iteration=i, reset=True)
  _, accuracy1 = m.train(20)

  if accuracy - accuracy1 > 0.1:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 36  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt35 , current checkpoint: ./ckpt36
INFO:tensorflow:Restoring parameters from ./ckpt35/model.ckpt-15
    average 100.00 % is preserved, 52 masks
    acc: 60.710->60.710, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
Training
Epoch 1, Overall loss = 2.27 and accuracy of 0.117
Epoch 2, Overall loss = 1.93 and accuracy of 0.234
Epoch 3, Overall loss = 1.84 and accuracy of 0.272
Epoch 4, Overall loss = 1.8 and accuracy of 0.288
Epoch 5, Overall loss = 1.69 and accuracy of 0.353
Epoch 6, Overall loss = 1.59 and accuracy of 0.401
Epoch 7, Overall loss = 1.52 and accuracy of 0.433
Epoch 8, Overall loss = 1.47 and accuracy of 0.458
Epoch 9, Overall loss = 1.42 and accuracy of 0.479
Epoch 10, Overall loss = 1.38 and accuracy of 0.494
Epoch 11, Overall loss = 1.35 and accuracy of 0.508
Epoch 12, Overall loss = 1.33 and accuracy of 0.517


In [ ]:
# %cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307
# m = Model()
# m.load_cifar10()
accuracy = 0.5501
m.definition('./ckpt36',accuracy, False, 0.8)

for i in range(37,38,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=2e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(18)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 37  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt36 , current checkpoint: ./ckpt37
INFO:tensorflow:Restoring parameters from ./ckpt36/model.ckpt-20
    average 81.86 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt36/model.ckpt-20
    (acc: 55.010->53.990) Increasimg preservation to 85.00 of parameters
    average 87.98 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt36/model.ckpt-20
    (acc: 55.010->54.310) Increasimg preservation to 90.00 of parameters
    average 91.88 % is preserved, 52 masks
    acc: 55.010->54.830, PRUNING ACCEPTED
Storing pruned graph, initializing, and restoring...
INFO:tensorflow:Restoring parameters from __tmp__/pruned_model.ckpt
Training
Epoch 1, Overall loss = 1.18 and accuracy of 0.571
Epoch 2, Overall loss = 1.16 and accuracy of 0.579
Epoch 3, Overall loss = 1.15 and accuracy of 0.583
Epoch 4, Overall loss = 1.14 and

In [ ]:
# %cd /content/gdrive/My\ Drive/UCLA/209AS\ AI\ on\ Chips/Project3/0307
# m = Model()
# m.load_cifar10()
accuracy = 0.5638
m.definition('./ckpt37',accuracy, False, 1.0)

for i in range(38,39,1):
  print('\n')
  print('<'*20, '\t    ROUND %d '%(i), '\t','>'*20)
  m.clear_graph()
  m.construct_model(lr=1e-3, ckpt_dir=True, iteration=i, reset=False)
  _, accuracy1 = m.train(15)

  if accuracy - accuracy1 > 0.05:
    print('\nRESULT: losing too much accuracy, abandon the result')
    m.check(accept=False)
  else:
    print('\nRESULT: pruning result accepted!', accuracy, '-->', accuracy1 )
    m.check()
    accuracy = accuracy1



<<<<<<<<<<<<<<<<<<<< 	    ROUND 38  	 >>>>>>>>>>>>>>>>>>>>
Clearing graph...
Constructing Model...
    >>> Model: last checkpoint: ./ckpt37 , current checkpoint: ./ckpt38
INFO:tensorflow:Restoring parameters from ./ckpt37/model.ckpt-18
    average 72.27 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt37/model.ckpt-18
    (acc: 56.380->53.120) Increasimg preservation to 75.00 of parameters
    average 76.34 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt37/model.ckpt-18
    (acc: 56.380->51.960) Increasimg preservation to 80.00 of parameters
    average 81.56 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt37/model.ckpt-18
    (acc: 56.380->54.230) Increasimg preservation to 85.00 of parameters
    average 87.60 % is preserved, 52 masks
INFO:tensorflow:Restoring parameters from ./ckpt37/model.ckpt-18
    (acc: 56.380->55.280) Increasimg preservation to 90.00 of parameters
    average 91.89 % is preserved, 52 m